In [1]:
from matplotlib import pyplot as plt
import numpy as np
import cmath
from scipy import interpolate
import pandas as pd
from ast import literal_eval


# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit

# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares

In [2]:
circ_consts = (3*10**(-8),0.35,619,50,10,0.0343,4.752*10**(-9),50,1.027*10**(-10),2.542*10**(-7),0,0,0,0)

L0 = circ_consts[0]
Rcoil = circ_consts[1]
R = circ_consts[2]
R1 = circ_consts[3]
r = circ_consts[4]
alpha = circ_consts[5]
beta1 = circ_consts[6]
Z_cable = circ_consts[7]
D = circ_consts[8]
M = circ_consts[9]
delta_C = circ_consts[10]
delta_phi = circ_consts[11]
delta_phase = circ_consts[12]
delta_l = circ_consts[13]

f = 32000000

g = 0.05
s = 0.04
bigy=(3-s)**0.5

k_range = 5000

scansize = .25
ranger = 0

In [3]:
def getArrayFromFunc(func,inputs):
    output = []
    for input in inputs:
        output.append((func(input)).real)
    return output

In [4]:
pi = np.pi
im_unit = complex(0,1)
sign = 1

def QCurve(w,U,knob,trim,eta,phi_const,Cstray):


    I = U*1000/R #Ideal constant current, mA

    #Derived quantities
    f = 32000000
    w_res = 2*pi*f
    w_low = 2 * pi * (213 - scansize) * (1e6)
    w_high = 2 * pi * (213 + scansize) * (1e6)
    delta_w = 2 * pi * 500 * ((1e3)/500)        

    #Functions
    def slope():
        return delta_C / (0.25 * 2 * pi * 1e6)

    def slope_phi():
        return delta_phi / (0.25 * 2 * pi * 1e6)

    def Ctrim(w):
        return slope()*(w - w_res)

    def Cmain():
        return 20*(1e-12)*knob

    def C(w):
        return Cmain() + Ctrim(w)*(1e-12)

    def Cpf():
        return C(w_res)*(1e12)

    #Derived quantities
    S = 2*Z_cable*alpha

    #Functions

    def Z0(w):
        return cmath.sqrt( (S + w*M*im_unit) / (w*D*im_unit))

    def beta(w):
        return beta1*w

    def gamma(w):
        return complex(alpha,beta(w))

    def ZC(w):
        if  w != 0 and C(w) != 0:
            return 1/(im_unit*w*C(w))
        return 1

    #More derived quantities
    vel = 1/beta(1)

    #More functions
    def lam(w):
        return vel/f

    #Even more derived quantities
    l_const = trim*lam(w_res)

    #Even more functions
    def l(w):
        return l_const + delta_l
    
        #Variables for creating splines
    k_ints = range(0,512)
    k = np.array(k_ints,float)
    x = (k*delta_w)+(w_low)
    Icoil_TE = Icoil = 0.11133
    # butxi = np.zeros(512)
    # butxii = np.zeros(512)
    # Icoil = np.empty(512); Icoil.fill(0.11133)
    # butxi = []
    # butxii = []
    vback = []
    vreal = []    
    # Icoil = []
    
    # for item in deriv_sig:
    #     butxi.append(item)
    # for item in main_sig:
    #     butxii.append(item)
    # for item in backgmd_sig:
    #     vback.append(item)
    # for item in backreal_sig:
    #     vreal.append(item)
    # for item in current_sig:
    #     Icoil.append(item)
    
    # x1 = interpolate.interp1d(x,butxi,fill_value=0.0,bounds_error=False)
    # x2 = interpolate.interp1d(x,butxii,fill_value=0.0,bounds_error=False)
    # b = interpolate.interp1d(x,vback,fill_value="extrapolate",kind="quadratic",bounds_error=False)
    # rb = interpolate.interp1d(x,vreal,fill_value="extrapolate",kind="quadratic",bounds_error=False)
    # ic = interpolate.interp1d(x,Icoil,fill_value="extrapolate",kind="linear",bounds_error=False)

    def chi(w):
        return complex(w,-1*w)



    def pt(w):
        # return ic(w)/Icoil_TE
        return float(1)

    def L(w):
        return L0*(1+(sign*4*pi*eta*pt(w)*chi(w)))

    def real_L(w):
        return L(w).real

    def imag_L(w):
        return L(w).imag

    def ZLpure(w):
        return im_unit*w*L(w) + Rcoil

    def Zstray(w):
        if w != 0 and Cstray !=0:
            return 1/(im_unit*w*Cstray)
        return 1

    def ZL(w):
        return ZLpure(w)*Zstray(w)/(ZLpure(w)+Zstray(w))

    def ZT(w):
        return Z0(w)*(ZL(w) + Z0(w)*np.tanh(gamma(w)*l(w)))/(Z0(w) + ZL(w)*np.tanh(gamma(w)*l(w)))


    def Zleg1(w):
        return r + ZC(w) + ZT(w)

    def Ztotal(w):
        return R1 / (1 + (R1 / Zleg1(w)) )

    #Adding parabolic term

    xp1 = w_low
    xp2 = w_res
    xp3 = w_high
    yp1 = 0
    yp2 = delta_phase
    yp3 = 0

    alpha_1 = yp1-yp2
    alpha_2 = yp1-yp3
    beta_1 = (xp1*xp1) - (xp2*xp2)
    beta_2 = (xp1*xp1) - (xp3*xp3)
    gamma_1 = xp1-xp2
    gamma_2 = xp1-xp3
    temp=(beta_1*(gamma_1/gamma_2) - beta_2)
    a= (gamma_2 *(alpha_1/gamma_1) - alpha_2)/temp
    bb = (alpha_2 - a*beta_2)/gamma_2
    c = yp1 - a*xp1*xp1 - bb*xp1

    def parfaze(w):
        return a*w*w + bb*w + c

    def phi_trim(w):
        return slope_phi()*(w-w_res) + parfaze(w)

    def phi(w):
        return phi_trim(w) + phi_const

    # def V_out(w):
    return (-1*(I*Ztotal(w)*np.exp(im_unit*phi(w)*pi/180))).real
    

    # larger_k = range(0,k_range)
    # larger_x = np.array(larger_k, float)
    # w_range = w_high - w_low
    # larger_range = (delta_w*larger_x)+(w_low-5*w_range)
    
    # out_y = getArrayFromFunc(V_out,x)
    # if (ranger == 1):
    #     out_y = getArrayFromFunc(V_out,larger_range)
    # return out_y

In [5]:
def Lineshape():
    
    def cosal(x,eps):
        return (1-eps*x-s)/bigxsquare(x,eps)
    
    def c(x):
        return ((g**2+(1-x-s)**2)**0.5)**0.5


    def bigxsquare(x,eps):
        return (g**2+(1-eps*x-s)**2)**0.5


    def mult_term(x,eps):
        return float(1)/(2*np.pi*np.sqrt(bigxsquare(x,eps)))


    def cosaltwo(x,eps):
        return ((1+cosal(x,eps))/2)**0.5


    def sinaltwo(x,eps):
        return ((1-cosal(x,eps))/2)**0.5


    def termone(x,eps):
        return np.pi/2+np.arctan((bigy**2-bigxsquare(x,eps))/((2*bigy*(bigxsquare(x,eps))**0.5)*sinaltwo(x,eps)))


    def termtwo(x,eps):
        return np.log((bigy**2+bigxsquare(x,eps)+2*bigy*(bigxsquare(x,eps)**0.5)*cosaltwo(x,eps))/(bigy**2+bigxsquare(x,eps)-2*bigy*(bigxsquare(x,eps)**0.5)*cosaltwo(x,eps)))

    return mult_term(x,eps)*(2*cosaltwo(x,eps)*termone(x,eps)+sinaltwo(x,eps)*termtwo(x,eps))



In [6]:
df = pd.read_csv('Deuteron.csv',delimiter=',')

f_values = df['freq_list'].apply(literal_eval)

baseline_values = df['baseline'].apply(literal_eval)

f = np.array(f_values[1])
V = np.array(baseline_values[1])



In [7]:
def chi_squared(U,knob,trim,eta,phi_const,Cstray):
    chi_squared = 0
    for i in range(len(f)):
        # Calculate the expected value using the baseline function A
        expected = QCurve(f[i],U,knob,trim,eta,phi_const,Cstray)
        
        # Calculate the residual
        residual = (V[i] - expected) 
        # since we don't have the uncertainties you can just consider the squared deviation
        # Accumulate the chi-squared value
        chi_squared += residual**2   
    return chi_squared
chi_squared(U = 0.1, knob= .126,trim= 1.5,eta= 0.0104,phi_const= 6.1319,Cstray= 10**(-15))


18646.321565387272

In [8]:
fit = Minuit(chi_squared,U = 0.1, knob= .126,trim= 1.5,eta= 0.0104,phi_const= 6.1319,Cstray= 10**(-15)
             limit_U = (0,0.2), limit_knob = (0,.4), limit_trim = (0,3), limit_eta = (0,.02), limit_phi_const = (2,7), limit_Cstray = (10**(-16), 10**(-14)))
fit.migrad()
print(fit.values) # to print the fit result
print(fit.errors)  # to print the fit parameters' errors
print(fit.fval) # to print the minimized value

E VariableMetricBuilder Initial matrix not pos.def.
E VariableMetricBuilder Initial matrix not pos.def.
E VariableMetricBuilder Initial matrix not pos.def.
<ValueView U=0.040896040584503066 knob=-188322.75701080868 trim=0.8109544610313054 eta=-9770203.36452626 phi_const=48.36771705451924 Cstray=1.744303362242127e-07>
<ErrorView U=0.0100916991622735 knob=42119.382640327276 trim=0.1785688157513129 eta=4079345.2620458375 phi_const=12.131998665699978 Cstray=1.6803827647930367e-06>
0.0006369396772923124
